In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
figer_path = base_dir + "FIGER/"

Mounted at /content/drive/


In [ ]:
import json

with open(figer_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [ ]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  1505765 elementi


In [ ]:
#ci sono 36 frasi che hanno le mentions vuote, le cancello
indici = []
for i,elem in enumerate(examples_train):
  if len(elem['mentions'])==0:
     indici.append(i)

pop=0
for num in indici:
  del examples_train[num-pop]
  pop+=1

In [ ]:
entire_dataset_list = iter(examples_train)

In [ ]:
lunghezze={
    'minori=_10':0,
    'maggiori_10_minori=_100':0,
    'maggiori_100_minori=200':0,
    'maggiori_200':0,
}

In [ ]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords

fp = open(path_corpus_data + 'corpus_data_entire_figer.txt', "w")
restricted_set = set()
mean=0
for num,i in enumerate(tqdm(entire_dataset_list)):
  
  #calcolo le permutazioni in modo grossolano e skippo le frasi che genererebbero più di 200 permutazioni
  prod = 1
  for elem in i['mentions']:
    prod*=len(elem['labels'])

  if prod<=10:
    lunghezze['minori=_10']+=1
  if prod>10 and  prod<=100:
    lunghezze['maggiori_10_minori=_100']+=1
  if prod>100 and prod<=200:
    lunghezze['maggiori_100_minori=200']+=1
  if (prod>200):
    lunghezze['maggiori_200']+=1
    continue

  mean+=prod
  istance = i['mentions'][:]
  if len(istance)>1:
    d = sorted(istance, key=operator.itemgetter("start"))
    d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}
    all_list=[]
    for key, value in d.items():
      all_list.append(value)

    permutations = list(itertools.product(*all_list))

    iter = 0
    for p in permutations:
          mini_list = []
          for lista_labels in p:
            mini_list.append(lista_labels['labels'])

          permutations_2 = itertools.product(*mini_list)

          for p_2 in permutations_2:
            phrase = i['tokens'][:]
            num_pop=0
            for h in range(0, len(p_2)):
              start = permutations[iter][h]['start']
              end = permutations[iter][h]['end']
              phrase[start-num_pop:end-num_pop] = [p_2[h]]
              if ((end-start)>1):
                num_pop += (end - start) - 1
              restricted_set.add(p_2[h])

            fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
          iter+=1
  else:
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(k)
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k]
        restricted_set.add(k)

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

fp.close()

1505729it [02:20, 10741.95it/s]


In [ ]:
lunghezze

{'maggiori_100_minori=200': 12385,
 'maggiori_10_minori=_100': 185841,
 'maggiori_200': 20574,
 'minori=_10': 1286929}

In [ ]:
print(f"media permutazioni {mean/len(examples_train)} frasi skippate {lunghezze['maggiori_200']}, frasi processate {len(examples_train)-lunghezze['maggiori_200']}")

media permutazioni 7.282461850704874 frasi skippate 20574, frasi processate 1485155


In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_figer.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)